In [3]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import load_model
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.decomposition import PCA
import pickle
import math
import gc
from collections import defaultdict

In [4]:
##### FUNCTIONS #####


def pairwise_combinations_with_ids_and_values(X):
    n_samples = X.shape[0]
    generated_pairs = set()  # To store generated pairs and avoid duplicates
    X_pairs = []
    X_pair_ids = []

    for i in range(n_samples):
        date_i = X[i, 0]
        same_date_indices = [j for j in range(i + 1, n_samples) if X[j, 0] == date_i]
        for j in same_date_indices:
            pair_key = tuple(sorted((i, j)))  # Create a unique key for the pair
            if pair_key not in generated_pairs:  # Check if pair is already generated
                X_pair = np.array([X[i, 2:], X[j, 2:]])
                X_pair_id = np.array([X[i, :2], X[j, :2]])
                generated_pairs.add(pair_key)
                X_pairs.append(X_pair)
                X_pair_ids.append(X_pair_id)

    return np.array(X_pairs, dtype='float32'), np.array(X_pair_ids)




def pairwise_generator5(X, y, batch_size):
    n_samples = X.shape[0]

    # Create a dictionary to store indices for each date
    date_indices = defaultdict(list)
    for i in range(n_samples):
        date_indices[X[i, 0]].append(i)

    while True:
        indices = np.arange(n_samples)
        np.random.shuffle(indices)  # Shuffle indices to create random batches
        batch_X = X[indices]
        batch_y = y[indices]

        X_batch = []
        y_batch = []
        generated_pairs = set()  # Reset generated pairs for each batch

        for i in range(n_samples):
            date_i = batch_X[i, 0]
            same_date_indices = date_indices[date_i]
            for j in same_date_indices:
                if i >= j:  # Ignore duplicate pairs and self-pairs
                    continue
                pair_key = tuple(sorted([i, j]))  # Use sorted indices as a unique key
                if pair_key not in generated_pairs:  # Check if pair is already generated
                    X_pair = np.array([batch_X[i, 2:], batch_X[j, 2:]])
                    y_pair = 1 if batch_y[i, 2] > batch_y[j, 2] else 0
                    X_batch.append(X_pair)
                    y_batch.append(y_pair)
                    generated_pairs.add(pair_key)

                if len(X_batch) == batch_size:
                    yield np.array(X_batch, dtype='float32'), np.array(y_batch, dtype='float32').reshape(-1,1)
                    X_batch, y_batch = [], []  # Reset batches

In [9]:
##### TRAIN #####
def train(X_train: pd.DataFrame, y_train: pd.DataFrame, model_directory_path: str = "../resources") -> None:
    max_date = X_train['date'].max()
    min_date = 150
    X_train_orig = X_train[(X_train['date'] < max_date) & (X_train['date'] > min_date)]
    X_test = X_train[X_train['date'] == max_date]
    y_train_orig = y_train[(y_train['date'] < max_date) & (y_train['date'] > min_date)]
    y_test = y_train[y_train['date'] == max_date]

    #Scaling
    scaler = StandardScaler()
    X_ids = np.asarray(X_train_orig[['date', 'id']])
    X_scale_pca = X_train_orig.drop(columns=['date', 'id'])
    X_scale_pca = scaler.fit_transform(X_scale_pca)


    X_test_ids = np.asarray(X_test[['date', 'id']])
    X_test_scale_pca = X_test.drop(columns=['date', 'id'])
    X_test_scale_pca = scaler.transform(X_test_scale_pca)


    X_train_concat = np.concatenate((X_ids, X_scale_pca), axis=1)
    y_train = np.asarray(y_train)

    X_test_concat = np.concatenate((X_test_ids, X_test_scale_pca), axis=1)
    y_test = np.asarray(y_test)


    #Save out Scaler and PCA
    with open(Path(model_directory_path) / 'scaler.pkl', 'wb') as file:
        pickle.dump(scaler, file)

    date_list = list(set(X_train_orig['date']))

    batch_size = 500
    train_generator = pairwise_generator5(X_train_concat, y_train, batch_size)
    test_generator= pairwise_generator5(X_test_concat, y_test, batch_size)

    print(X_train_concat.shape)


    #Model Training
    model_pathname = Path('../resources') / "model.keras"

    if model_pathname.is_file():
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=20,
            verbose=0,
            mode='auto',
            baseline=None)

        history = model.fit(
            train_generator,
            batch_size=batch_size,
            steps_per_epoch = 1000,
            epochs=30,
            validation_data=test_generator,
            validation_steps = 500,
            callbacks=[mc, early_stopping],
            shuffle=True,
            use_multiprocessing=False,
            verbose=1
        )

    else:
        #Neural Network Model
        mc = ModelCheckpoint(model_pathname, monitor='val_loss', mode='min', verbose=1, save_best_only=False)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=20,
            verbose=1,
            mode='auto',
            baseline=None)

        model = keras.Sequential([
            keras.layers.Dense(200, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer=keras.regularizers.l2(0.01), input_shape=(2, (X_train_concat.shape[1] - 2))),
            #keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.5),  # Adding dropout regularization
            keras.layers.Dense(100, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer=keras.regularizers.l2(0.01)),
            #keras.layers.BatchNormalization(),
            # keras.layers.Dropout(0.5),  # Adding dropout regularization
            # keras.layers.Dense(250, activation='relu', kernel_initializer='lecun_normal'),
            # keras.layers.BatchNormalization(),
            # keras.layers.Dropout(0.5),  # Adding dropout regularization
            # keras.layers.Dense(100, activation='relu', kernel_initializer='lecun_normal'),
            # keras.layers.BatchNormalization(),
            keras.layers.Dropout(0.5),  # Adding dropout regularization
            keras.layers.Dense(25, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer=keras.regularizers.l2(0.01)),
            keras.layers.Dropout(0.5),  # Adding dropout regularization,
            keras.layers.Flatten(),
            keras.layers.Dense(1, activation='sigmoid', kernel_initializer='lecun_normal')
        ])

        optimizer = keras.optimizers.Adam(learning_rate=0.001)

        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(
            train_generator,
            batch_size=batch_size,
            steps_per_epoch=1000,
            epochs=30,
            validation_data=test_generator,
            validation_steps=500,
            callbacks=[mc, early_stopping],
            shuffle=True,
            use_multiprocessing=False,
            verbose=1
        )

        model.save(model_pathname)




    gc.collect()

    print("Finished All Training")

In [11]:
def infer(X_test: pd.DataFrame, model_directory_path: str = "../resources") -> pd.DataFrame:
    X_test_orig = X_test.copy()

    # Load Scaler
    with open(Path(model_directory_path) / 'scaler.pkl', 'rb') as file:
        scaler = pickle.load(file)


    # Scaling
    X_ids = np.asarray(X_test_orig[['date', 'id']])
    X_scale_pca = X_test_orig.drop(columns=['date', 'id'])
    X_scale_pca = scaler.transform(X_scale_pca)

    X_test_concat = np.concatenate((X_ids, X_scale_pca), axis=1)


    result_df = pd.DataFrame(columns=['date', 'id', 'value'])

    # Load Model
    model_pathname = Path(model_directory_path) / "model.keras"
    model = load_model(model_pathname)

    # Pairwise Transformation using the pairwise generator
    batch_size = 1000
    X_test, X_test_ids = pairwise_combinations_with_ids_and_values(X_test_concat)

    print("Predicting for Test Data")
    preds = model.predict(X_test)

    preds_df = pd.DataFrame({'id': X_test_ids[:, 0, 1].flatten(), 'date': X_test_ids[:, 0, 0].flatten(), 'value': preds.flatten()})
    preds_df = preds_df.groupby(['date', 'id']).mean().reset_index()

    result_df = pd.merge(X_test_orig, preds_df, on=['id', 'date'], how='left')
    result_df['value'] = result_df['value'].fillna(result_df['value'].mean())

    minmax = MinMaxScaler(feature_range=(-1, 1))
    result_df['value'] = minmax.fit_transform(result_df[['value']])

    print("Finished predicting Test Data")

    return result_df

In [12]:
X_train = pd.read_parquet('../data/X_train.parquet')
y_train = pd.read_parquet('../data/y_train.parquet')
X_test = pd.read_parquet('../data/X_test.parquet')

In [13]:
train(X_train, y_train)

(415591, 463)
Epoch 1/30
 999/1000 [============================>.] - ETA: 0s - loss: 1.4253 - accuracy: 0.5991
Epoch 1: saving model to ..\resources\model.keras
1000/1000 [==============================] - 38s 36ms/step - loss: 1.4249 - accuracy: 0.5988 - val_loss: 0.7968 - val_accuracy: 0.5229
Epoch 2/30
1000/1000 [==============================] - ETA: 0s - loss: 0.7569 - accuracy: 0.6007
Epoch 2: saving model to ..\resources\model.keras
1000/1000 [==============================] - 33s 33ms/step - loss: 0.7569 - accuracy: 0.6007 - val_loss: 0.7692 - val_accuracy: 0.4931
Epoch 3/30
 998/1000 [============================>.] - ETA: 0s - loss: 0.7263 - accuracy: 0.5656
Epoch 3: saving model to ..\resources\model.keras
1000/1000 [==============================] - 33s 33ms/step - loss: 0.7262 - accuracy: 0.5658 - val_loss: 0.7283 - val_accuracy: 0.5070
Epoch 4/30
 999/1000 [============================>.] - ETA: 0s - loss: 0.7061 - accuracy: 0.5273
Epoch 4: saving model to ..\resources\m

KeyboardInterrupt: 

In [ ]:
results = infer(X_test)